In [85]:
%pip install nbformat
%pip install imbalanced-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [86]:
import nbformat

# Read your notebook (assuming version 4 for example purposes)
nb = nbformat.read("ml_project.ipynb", as_version=4)

# Normalize the notebook to add missing id fields and other updates
nbformat.validator.validate(nb)

# Write the normalized notebook back to a file
nbformat.write(nb, "ml_project_normalized.ipynb")

In [87]:
%run ml_project_normalized.ipynb

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
dropped columns
cleaned intake time
cleaned intake condition
cleaned age and sex
cleaned breed
cleaned color
dropped columns


/var/folders/yf/n0zzld6945vfyf6hj9_n1g4c0000gn/T/ipykernel_65874/4020467989.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dt_series = pd.to_datetime(df['intake_time'], errors='coerce')


cleaned intake time
cleaned intake condition
cleaned age and sex
cleaned breed
cleaned color
Done running ml_project.ipynb.


In [88]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from imblearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, cross_val_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import make_scorer
import numpy as np
import pandas as pd
from xgboost import XGBClassifier

def train_classifier(X_train, y_train, X_test, rare_classes, cat_cols):
    """
    Trains an XGBoost model using hyperparameter tuning.
    Only the categorical columns in cat_cols will be one-hot encoded.
    
    Parameters:
    X_train (pd.DataFrame): Training features.
    y_train (pd.Series or np.array): Training target values.
    X_test (pd.DataFrame): Test features.
    rare_classes (list): List of integer-encoded classes to be oversampled.
    cat_cols (set): Set of column names of categorical features to be one-hot encoded.
    
    Returns:
        best_estimator: The best estimator from RandomizedSearchCV.
        test_predictions: The predicted labels for X_test from the best estimator.
    """
    
    class_labels = np.unique(y_train)
    class_weights = compute_class_weight('balanced', classes=class_labels, y=y_train)
    class_weight_dict = dict(zip(class_labels, class_weights))
    sample_weights = np.array([class_weight_dict[y] for y in y_train])
    
    # Determine categorical columns to be transformed (ensuring they're in X_train).
    categorical_features = [col for col in X_train.columns if col in cat_cols]
    
    # Create a ColumnTransformer that one-hot encodes only the categorical columns.
    preprocessor = ColumnTransformer(
        transformers=[
            ("onehot", OneHotEncoder(handle_unknown='ignore', sparse_output=True), categorical_features)
        ],
        remainder='passthrough'
    )
    
    # Define the pipeline with the preprocessor and classifier.
    pipeline = Pipeline([
        ("preprocessor", preprocessor),
        ("xgb", XGBClassifier(eval_metric='logloss', verbosity=1))
    ])
    
    # Hyperparameter grid for RandomizedSearchCV.
    param_distributions = {
        "xgb__max_depth": [2, 3, 5, 7, 9],
        "xgb__learning_rate": [0.01, 0.05, 0.1, 0.2, 0.3],
        "xgb__n_estimators": [50, 100, 200, 300, 500],
        "xgb__subsample": [0.3, 0.5, 0.7, 1.0],
        "xgb__colsample_bytree": [0.3, 0.5, 0.7, 1.0],
        "xgb__min_child_weight": [1, 3, 5, 7],
        "xgb__gamma": [0, 0.1, 0.3, 0.5, 1.0],
        "xgb__reg_alpha": [0, 0.01, 0.1, 0.5, 1.0],
        "xgb__reg_lambda": [0.5, 1.0, 1.5, 2.0, 3.0]
    }
    
    # Set up stratified cross-validation.
    stratified_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    randomized_search = RandomizedSearchCV(
        estimator=pipeline,
        param_distributions=param_distributions,
        n_iter=100,
        cv=stratified_cv,
        scoring='balanced_accuracy',
        verbose=3,
        random_state=42,
    )
    
    print(f"\n[INFO] Starting training with {len(X_train)} samples and {len(y_train)} labels")
    
    # Fit the model with hyperparameter search.
    randomized_search.fit(X_train, y_train, **{'xgb__sample_weight': sample_weights})
    print(f"[INFO] Training complete. Best model fitted on {len(X_train)} samples.\n")
    
    # Retrieve and display one-hot encoded features for inspection.
    best_preprocessor = randomized_search.best_estimator_.named_steps['preprocessor']
    encoded_X_train = best_preprocessor.transform(X_train)
    # Convert sparse output to dense format if necessary.
    if hasattr(encoded_X_train, "toarray"):
        encoded_X_train = encoded_X_train.toarray()
    feature_names = best_preprocessor.get_feature_names_out()
    X_train_encoded_df = pd.DataFrame(encoded_X_train, columns=feature_names)
    print("Sample of transformed training features:")
    print(X_train_encoded_df.head())
    
    print('Best parameters:', randomized_search.best_params_)
    print('Best cross-validation accuracy:', randomized_search.best_score_)
    
    # Evaluate generalization using cross_val_score.
    # (Assuming classification_report_with_accuracy_score is defined elsewhere.)
    cv_scores = cross_val_score(
        randomized_search.best_estimator_,
        X_train,
        y_train,
        cv=5,
        verbose=3,
        scoring='balanced_accuracy'
    )
    print('Generalization accuracy (via cross_val_score):', cv_scores.mean())
    
    # Predict on test data using the best estimator.
    test_predictions = randomized_search.predict(X_test)
    
    return randomized_search.best_estimator_, test_predictions

In [89]:
# from sklearn.utils import resample
# df_train_downsample = resample(df_train, replace=True, n_samples=10000, random_state=42)
# print(df_train_downsample.shape)
df_train.head()

,intake_type,intake_condition,animal_type,sex_upon_intake,age_upon_intake,breed,intake_year,intake_month,intake_hour,is_mix,primary_color,outcome_type
0,Stray,Normal / Behavior,Dog,Spayed Female,8.000000,English Springer Spaniel,2015,7,12,0,white,Return to Owner
1,Stray,Normal / Behavior,Dog,Intact Male,0.916667,Basenji,2016,4,18,1,sable,Return to Owner
2,Public Assist,Normal / Behavior,Cat,Neutered Male,2.000000,Domestic Shorthair,2022,5,0,0,orange,Transfer
3,Owner Surrender,Normal / Behavior,Dog,Neutered Male,2.000000,Labrador Retriever,2017,2,12,1,chocolate,Return to Owner
4,Public Assist,Normal / Behavior,Dog,Neutered Male,6.000000,Great Dane,2019,4,9,1,black,Return to Owner


In [90]:
# Encode the target variable.
le = LabelEncoder()
X_train = df_train.drop(columns=['outcome_type'])
y_train = df_train['outcome_type']
y_train = le.fit_transform(y_train)
print('Encoding mapping:', le.classes_)

# Identify rare classes that need oversampling.
rare_classes = [
    label for label, count in pd.Series(y_train).value_counts().items()
    if count < 0.05 * len(y_train)
]
print("Rare classes:")
for cls in rare_classes:
    print(f"  {cls}: {le.classes_[cls]}")

# Define column groups (this example will one-hot encode all columns, so cat_cols and num_cols are not used in the transformer).
cat_cols = {'intake_type', 'intake_condition', 'animal_type', 'sex_upon_intake', 'breed', 'primary_color', 'intake_month', 'intake_hour', 'intake_year'}
categorical_features = [col for col in X_train.columns if col in cat_cols]

# Train the classifier with the refactored pipeline.
best_model, test_predictions = train_classifier(
    X_train=X_train,
    y_train=y_train,
    X_test=df_test,
    rare_classes=rare_classes,
    cat_cols=categorical_features
)

# Convert predictions back to original labels.
predictions = le.inverse_transform(test_predictions)

# Save predictions; assumes save_predictions is defined elsewhere.
save_predictions(predictions, 'xg_boost')

Encoding mapping: ['Adoption' 'Died' 'Euthanasia' 'Return to Owner' 'Transfer']
Rare classes:
  2: Euthanasia
  1: Died

[INFO] Starting training with 111155 samples and 111155 labels
Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV 1/5] END xgb__colsample_bytree=0.3, xgb__gamma=0.3, xgb__learning_rate=0.1, xgb__max_depth=2, xgb__min_child_weight=7, xgb__n_estimators=500, xgb__reg_alpha=0.1, xgb__reg_lambda=3.0, xgb__subsample=0.7;, score=0.532 total time=   3.8s
[CV 2/5] END xgb__colsample_bytree=0.3, xgb__gamma=0.3, xgb__learning_rate=0.1, xgb__max_depth=2, xgb__min_child_weight=7, xgb__n_estimators=500, xgb__reg_alpha=0.1, xgb__reg_lambda=3.0, xgb__subsample=0.7;, score=0.524 total time=   4.4s
[CV 3/5] END xgb__colsample_bytree=0.3, xgb__gamma=0.3, xgb__learning_rate=0.1, xgb__max_depth=2, xgb__min_child_weight=7, xgb__n_estimators=500, xgb__reg_alpha=0.1, xgb__reg_lambda=3.0, xgb__subsample=0.7;, score=0.520 total time=   4.1s
[CV 4/5] END xgb__colsample_bytree=0